<a href="https://colab.research.google.com/github/Shakil018/ML_Project/blob/main/CODES/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To access the dataset in drive from google collab
# First: upload the dataset directory in drive. then run the follwing two lines 

# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# pip install ImageDataset
!pip install image_dataset_loader
!pip install scikeras
import image_dataset_loader
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import cross_val_score, KFold
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
from functools import partial





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# # Loading the 17 class dataset:



# Random rotation from 15 to 15, rotations of
# multiple of 90 at random, random distortion, shear transformation,
# vertical flip, horizontal flip, skewing and intensity
# transformation

# another approach:


from keras.preprocessing.image import ImageDataGenerator

# specify the path to the image directory

image_dir_symptom = "/content/drive/MyDrive/L4 - T2/CSE - 472/PROJECT/DATASET/SYMPTOM_Resized"
image_dir_original = "/content/drive/MyDrive/L4 - T2/CSE - 472/PROJECT/DATASET/Resized_Original_Data"



# set up an ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale=1./255)

# use the flow_from_directory method to load the images and labels
batch_size = 64
image_size = (224, 224) # adjust the image size as needed
generator_symptom = datagen.flow_from_directory(
    image_dir_symptom,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

generator_original = datagen.flow_from_directory(
    image_dir_original,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)




# extract the class names from the generator
class_names_symptom = list(generator_symptom.class_indices.keys())
class_names_original = list(generator_original.class_indices.keys())


# get the number of classes
num_classes_symptom = len(class_names_symptom)
num_classes_original = len(class_names_original)


# get the number of samples in the dataset
num_samples_symptom = generator_symptom.samples
num_samples_original = generator_original.samples




Found 1426 images belonging to 17 classes.
Found 1426 images belonging to 9 classes.


In [ ]:
# print some information about the dataset
print("Symptom Dataset:")
print(f"Found {num_samples_symptom} images belonging to {num_classes_symptom} classes.")
print(f"The class names are: {class_names_symptom}.")


# print some information about the dataset
print("Original Dataset: ")
print(f"Found {num_samples_original} images belonging to {num_classes_original} classes.")
print(f"The class names are: {class_names_original}.")



Symptom Dataset:
Found 1426 images belonging to 17 classes.
The class names are: ['BLB1', 'BPH1', 'BPH2', 'Brown_Spot1', 'False_Smut1', 'False_Smut2', 'Healthy1', 'Healthy2', 'Healthy3', 'Hispa1', 'Hispa2', 'Neck_Blast1', 'Sheath_Blight_Rot1', 'Sheath_Blight_Rot2', 'Sheath_Blight_Rot3', 'Stemborer1', 'Stemborer2'].
Original Dataset: 
Found 1426 images belonging to 9 classes.
The class names are: ['BLB', 'BPH', 'Brown_Spot', 'False_Smut', 'Healthy_Plant', 'Hispa', 'Neck_Blast', 'Sheath_Blight_Rot', 'Stemborer'].


In [ ]:
# Symptom dataset X, y:

# set the number of steps per epoch
steps_per_epoch = num_samples_symptom // batch_size

# get the training data
X_train_symptom = []
y_train_symptom = []

for i in range(steps_per_epoch+1):
    # get a batch of images and labels
    batch = next(generator_symptom)
    # append the images and labels to the training data lists
    X_train_symptom.append(batch[0])
    y_train_symptom.append(batch[1])


# concatenate the batches into a single numpy array
X_train_symptom = np.concatenate(X_train_symptom)
y_train_symptom = np.concatenate(y_train_symptom)

print(y_train_symptom[0])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# Symptom dataset X, y:

# set the number of steps per epoch
steps_per_epoch = num_samples_original // batch_size

# get the training data
X_train_original = []
y_train_original = []

for i in range(steps_per_epoch+1):
    # get a batch of images and labels
    batch = next(generator_original)
    # append the images and labels to the training data lists
    X_train_original.append(batch[0])
    y_train_original.append(batch[1])


# concatenate the batches into a single numpy array
X_train_original = np.concatenate(X_train_original)
y_train_original = np.concatenate(y_train_original)

print(y_train_original[0])


[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:


def create_model(num_of_classes, learning_rate = 0.0001):
    # model = Sequential()
    # model.add(Dense(10, input_dim=8, activation='relu'))
    # model.add(Dense(1, activation='sigmoid'))
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # return model


    # Create a sequential model
    model = tf.keras.models.Sequential()

    # Conv block 1

    # Add a 2D convolution layer with 32 filters and a kernel size of 3x3
    model.add(tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
    # Add batch normalization
    model.add(tf.keras.layers.BatchNormalization())
    # Add ReLU activation
    # model.add(tf.keras.layers.Activation('relu'))
    # Add a max pooling layer with a pool size of 2x2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


    # Conv block 2

    # Add a 2D convolution layer with 24 filters and a kernel size of 3x3
    model.add(tf.keras.layers.Conv2D(24, kernel_size=(3,3), activation='relu'))
    # Add batch normalization
    model.add(tf.keras.layers.BatchNormalization())
    # Add ReLU activation
    # model.add(tf.keras.layers.Activation('relu'))
    # Add a max pooling layer with a pool size of 2x2
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))



    # Conv block 3

    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


    # Conv block 4

    model.add(tf.keras.layers.Conv2D(48, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


    # Conv block 5

    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


    # Flattening
    model.add(tf.keras.layers.Flatten())



    # Add another dense layer with 24 neurons and ReLU activation
    # model.add(tf.keras.layers.Dense(128, activation='relu'))

    # Add a dense layer with 128 neurons and ReLU activation
    model.add(tf.keras.layers.Dense(128))

    # Add dropout with a rate of 0.3
    model.add(tf.keras.layers.Dropout(0.3))

    # Add ReLU activation
    model.add(tf.keras.layers.Activation('relu'))


    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Activation('relu'))

    # Add an output layer with a single neuron and sigmoid activation
    model.add(tf.keras.layers.Dense(num_of_classes))
    model.add(tf.keras.layers.Activation('softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model




In [ ]:
# Hyperparameters:

num_of_symptom_classes = 17
num_of_original_classes = 9

num_of_epochs = 10

learning_rate = 0.0001

In [ ]:
# Train the model in stage-1

model = create_model(num_of_symptom_classes, learning_rate)
num_of_epochs = 10

model.fit(X_train_symptom, y_train_symptom, epochs=num_of_epochs, batch_size=batch_size)


Epoch 1/10
23/23 [==============================] - 100s 4s/step - loss: 3.1005 - accuracy: 0.1192
Epoch 2/10
23/23 [==============================] - 104s 4s/step - loss: 2.3863 - accuracy: 0.2959
Epoch 3/10
23/23 [==============================] - 99s 4s/step - loss: 2.0786 - accuracy: 0.3752
Epoch 4/10
23/23 [==============================] - 102s 4s/step - loss: 1.8734 - accuracy: 0.4383
Epoch 5/10
23/23 [==============================] - 105s 5s/step - loss: 1.6592 - accuracy: 0.5084
Epoch 6/10
23/23 [==============================] - 106s 5s/step - loss: 1.5537 - accuracy: 0.5351
Epoch 7/10
23/23 [==============================] - 100s 4s/step - loss: 1.3984 - accuracy: 0.5926
Epoch 8/10
23/23 [==============================] - 101s 4s/step - loss: 1.2740 - accuracy: 0.6094
Epoch 9/10
23/23 [==============================] - 104s 4s/step - loss: 1.1465 - accuracy: 0.6704
Epoch 10/10
23/23 [==============================] - 99s 4s/step - loss: 1.0482 - accuracy: 0.6992


In [ ]:
# Saving the model weights:

model.save_weights('model_weights.h5')

# model.save('./MyModel_tf',save_format='tf')



In [ ]:

# Load the pretrained model into similar structure for stage-2:
pretrained_model = create_model(num_of_symptom_classes, learning_rate)

pretrained_model.load_weights("model_weights.h5")

# pretrained_model.summary()

# index = 0
# for layer in pretrained_model.layers:
#     index += 1
#     weights = layer.get_weights() # list of numpy arrays
#     print("Layer: ", index)
#     print(weights)



In [ ]:
# Create a new model by extracting layers from the original model:

# Removing the last two layers of shape(None, 17) and replacing it with stage-2 final layer of shape(None, 9)
extracted_layers = pretrained_model.layers[:-2]

# print(extracted_layers)

extracted_layers.append(tf.keras.layers.Dense(num_of_original_classes))
extracted_layers.append(tf.keras.layers.Activation('softmax'))

model_original = keras.Sequential(extracted_layers)
# model_original.summary()


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model_original.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# Train the pre-initialized model on original dataset:

model_original.fit(X_train_original, y_train_original, epochs=num_of_epochs, batch_size=batch_size)

# score = model.evaluate(X, Y, verbose=0)


Epoch 1/10
23/23 [==============================] - 105s 4s/step - loss: 2.2331 - accuracy: 0.2398
Epoch 2/10
 1/23 [>.............................] - ETA: 1:26 - loss: 1.6967 - accuracy: 0.4062

KeyboardInterrupt: ignored

In [ ]:
# Saving the final model:

model_original.save_weights('final_model_weights.h5')


In [ ]:
# what if it directly ran in stage-2 without pre-training in stage 1:

model_original_no_pretraining = create_model(num_of_original_classes, learning_rate)


model_original_no_pretraining.fit(X_train_original, y_train_original, epochs=num_of_epochs, batch_size=batch_size)


Epoch 1/10
23/23 [==============================] - 102s 4s/step - loss: 2.3183 - accuracy: 0.2020
Epoch 2/10
23/23 [==============================] - 104s 4s/step - loss: 1.7751 - accuracy: 0.3696
Epoch 3/10
23/23 [==============================] - 100s 4s/step - loss: 1.5582 - accuracy: 0.4558
Epoch 4/10
23/23 [==============================] - 103s 4s/step - loss: 1.3475 - accuracy: 0.5323
Epoch 5/10
23/23 [==============================] - 102s 4s/step - loss: 1.1916 - accuracy: 0.5961
Epoch 6/10
23/23 [==============================] - 102s 4s/step - loss: 1.0620 - accuracy: 0.6466
Epoch 7/10
23/23 [==============================] - 102s 4s/step - loss: 0.9655 - accuracy: 0.6683
Epoch 8/10
23/23 [==============================] - 100s 4s/step - loss: 0.8881 - accuracy: 0.7104
Epoch 9/10
23/23 [==============================] - 101s 4s/step - loss: 0.8186 - accuracy: 0.7405
Epoch 10/10
23/23 [==============================] - 103s 4s/step - loss: 0.7208 - accuracy: 0.7784


In [ ]:
# weights = model_original.get_weights() # returs a numpy list of weights
# print(weights)


# index = 0
# for layer in model_original.layers:
#     index += 1
#     weights = layer.get_weights() # list of numpy arrays
#     print("Layer: ", index)
#     print(weights)



# index = 0
# for layer_symptom, layer_original in zip(model_original.layers, pretrained_model.layers):
#     index += 1
#     weights_symptom = layer_symptom.get_weights() # list of numpy arrays
#     weights_original = layer_original.get_weights() # list of numpy arrays

#     # print(weights_sum)

#     if np.array_equal(weights_symptom, weights_original):
#       print("they are equal: ", index)
#     else:
#       print("not equal: ", index)

# pretrained_model.summary()

    # print("symptom: ", np.array(weights_symptom).shape)
    # print("original: ", np.array((weights_original)).shape)

    # print("Layer: ", index)
    # print(weights)

In [ ]:
# Determinig 10-fold cross validation accuracy: takes very long time
# test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)



# Create the KerasClassifier model
model = KerasClassifier(model=partial(create_model, learning_rate=learning_rate, num_of_classes=num_of_symptom_classes),
                        epochs=num_of_epochs, batch_size=batch_size, verbose=1)

# Define the cross-validation method
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Get the cross-validation scores
scores = cross_val_score(model, X_train_symptom, y_train_symptom, cv=kf)

# Print the mean and standard deviation of the scores
print(f'Accuracy: {np.mean(scores)} (+/- {np.std(scores)})')


Epoch 1/10
21/21 [==============================] - 93s 4s/step - loss: 2.9106 - accuracy: 0.1481
Epoch 2/10
21/21 [==============================] - 108s 5s/step - loss: 2.3867 - accuracy: 0.2564
Epoch 3/10
21/21 [==============================] - 103s 5s/step - loss: 2.1355 - accuracy: 0.3383
Epoch 4/10
21/21 [==============================] - 99s 5s/step - loss: 1.9527 - accuracy: 0.3843
Epoch 5/10
21/21 [==============================] - 104s 5s/step - loss: 1.7964 - accuracy: 0.4645
Epoch 6/10
21/21 [==============================] - 88s 4s/step - loss: 1.6364 - accuracy: 0.5160
Epoch 7/10
21/21 [==============================] - 94s 4s/step - loss: 1.5351 - accuracy: 0.5448
Epoch 8/10
21/21 [==============================] - 91s 4s/step - loss: 1.4438 - accuracy: 0.5651
Epoch 9/10
21/21 [==============================] - 94s 5s/step - loss: 1.3142 - accuracy: 0.6243
Epoch 10/10
3/3 [==============================] - 3s 688ms/step
Epoch 1/10
21/21 [==============================] 

KeyboardInterrupt: ignored

In [ ]:
# 10-fold cross validation accuracy: without including batch size:

model = create_model(num_of_symptom_classes, learning_rate)
num_of_epochs = 10

# Define the cross-validation method
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# Perform cross-validation

scores = cross_val_score(model, X_train_symptom, y_train_symptom, scoring='accuracy', cv=cv, epochs=num_of_epochs)

print(f'Accuracy scores for each fold: {scores}')

In [ ]:
# Make predictions with the model
# y_pred = model.predict(x_test)


In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
# !pip install gdown

# import gdown

# # Download the dataset from Google Drive
# gdown.download('https://drive.google.com/drive/folders/1nlyz1UMbcByhkqH-l2ihO9net1BYZnct?usp=sharing', 'ml_project/Resized_Original_Data.zip', quiet=False)

# # Unzip the dataset
# !unzip ml_project/Resized_Original_Data.zip


# New Section